# Functional Closures and Decorators

Sometimes we may have "cross-cutting" concerns: functionality which is not related to the "core" functionality of a function:

- logging
- authentication
- locking
- entry of the function into a global registry

Python has a syntax that uses two features of the language (scoping and first-class functions) to enable such operations to be separated from the core function.

First, though, let's see if you can guess what the following code prints:

In [ ]:
a = 5
b = 12
def foo():
    a = 10
    b = 15
    print(f'1 => a={a}, b={b}')  # '1 => a={}, b={}'.format(a, b)   Py 3.5+
    def bar():
        a = 20
        print(f'2 => a={a}, b={b}')
    bar()
    print(f'3 => a={a}, b={b}')
foo()
print(f'4 => a={a}')
print(f'5 => b={b}')

##  LEGB
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing (i.e. `nonlocal`)
  * G = global (i.e. `global`)
  * B = builtin (e.g., len() function)
  
```python
a = 'global scope'

def outer_func():
    b = 'local to outer_func()'
    def inner_func():
        c = 'local to inner_func()'
        print(b, 'enclosing/nonlocal scope')
        print(a, 'global scope')
```
                
* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which "remembers" a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

# Closures

In [1]:
def make_adder(x):
    
    def adder(y):
        return x + y # Python uses LEGB to find 'x'
    
    return adder

In [2]:
add39 = make_adder(x=39)
add39

<function __main__.make_adder.<locals>.adder(y)>

In [3]:
add39(10)

49

In [4]:
add12 = make_adder(12)
add12(10)

22

In [5]:
add39(10)

49

In [6]:
cell = add39.__closure__[0]

In [7]:
cell.cell_contents

39

In [8]:
x = make_adder(39)
x(10)

49

In [9]:
make_adder(39)(10)  # like add(39, 10)    

49

In [10]:
add6 = make_adder(6)

In [11]:
print(add6(10), add39(10))

16 49


In [12]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
#type(add39)
repr(add39)

'<function make_adder.<locals>.adder at 0x10c9d3940>'

In [13]:
repr(add6)

'<function make_adder.<locals>.adder at 0x10c9fc280>'

* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [14]:
def logging(function):
    print('Wrapping {}'.format(function))
    def wrapper(*args, **kwargs):
        print('Calling %r(%r, %r)' % (function, args, kwargs))
        return function(*args, **kwargs)
    return wrapper

In [15]:
def myprinter(x):
    print('The value of x is', x)

logging_myprinter = logging(myprinter)

Wrapping <function myprinter at 0x10c9fc310>


In [16]:
myprinter(42)

The value of x is 42


In [17]:
logging_myprinter(42)

Calling <function myprinter at 0x10c9fc310>((42,), {})
The value of x is 42


In [18]:
logging_myprinter

<function __main__.logging.<locals>.wrapper(*args, **kwargs)>

In [19]:
logging_myprinter(5)

Calling <function myprinter at 0x10c9fc310>((5,), {})
The value of x is 5


In [20]:
logging_myprinter.__closure__[0].cell_contents

<function __main__.myprinter(x)>

In [21]:
myprinter = logging(myprinter)

Wrapping <function myprinter at 0x10c9fc310>


In [22]:
myprinter(5)

Calling <function myprinter at 0x10c9fc310>((5,), {})
The value of x is 5


In [23]:
myprinter(22)

Calling <function myprinter at 0x10c9fc310>((22,), {})
The value of x is 22


## Decorators
* Wrapper functions are so common, that Python has its own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [24]:
def document_it(func):
    # below is a nested, or inner function
    def wrapper(*args, **kwargs):
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return wrapper

In [25]:
def add_ints(a, b):
    return a + b

print('Running plain old add_ints()')
print(add_ints(3, 5))

Running plain old add_ints()
8


In [26]:
# manual decorator assignment
cooler_add_ints = document_it(add_ints) 

print('Running cooler add_ints()')
cooler_add_ints(3, 5)

Running cooler add_ints()
Running function: add_ints
Positional arguments: (3, 5)
Keyword arguments: {}
Result: 8


8

In [27]:
def add_ints(a, b):
    return a + b
add_ints = document_it(add_ints)

In [28]:
add_ints(5, 8)

Running function: add_ints
Positional arguments: (5, 8)
Keyword arguments: {}
Result: 13


13

In [29]:
# decorator shorthand for what we did above
@document_it
def add_ints(a, b):
    return a + b

In [30]:
add_ints(4, b=7)

Running function: add_ints
Positional arguments: (4,)
Keyword arguments: {'b': 7}
Result: 11


11

In [31]:
add_ints

<function __main__.document_it.<locals>.wrapper(*args, **kwargs)>

Not all decorators wrap. 

In [32]:
# Flask-like routing

routes = {}  # map url => python function


def route(path):   # "decorator factory" (returns a decorator)
    print(f'Build route({path})')
    def decorator(function):   # decorator (takes a single function as input)
        print(f'Call decorator({path})({function})')
        routes[path] = function
        # (no wrapping going on here)
        return function
    return decorator

In [33]:
@route('/')
# @route('/foo')
def get_home():
    return 'Welcome'

@route('/about')
def get_about():
    return 'About us'

@route('/error')
@route('/other_error')
def get_error():
    return '404'

# route_error_decorator = route('/error')
# route_other_error_decorator = route('/other_error')
# def get_error():
#     return '404'
# get_error = route_other_error_decorator(get_error)
# get_error = route_error_decorator(get_error)

Build route(/)
Call decorator(/)(<function get_home at 0x10c9fc670>)
Build route(/about)
Call decorator(/about)(<function get_about at 0x10c9fcee0>)
Build route(/error)
Build route(/other_error)
Call decorator(/other_error)(<function get_error at 0x10ca0f3a0>)
Call decorator(/error)(<function get_error at 0x10ca0f3a0>)


In [34]:
routes

{'/': <function __main__.get_home()>,
 '/about': <function __main__.get_about()>,
 '/other_error': <function __main__.get_error()>,
 '/error': <function __main__.get_error()>}

In [35]:
def run_request(routes, url):
    function_to_call = routes[url]
    return function_to_call()

In [36]:
run_request(routes, '/')

'Welcome'

In [37]:
run_request(routes, '/about')

'About us'

In [38]:
run_request(routes, '/error')

'404'

```python
@EXPR0  # (
@EXPR1  # (
def FUNC():
    ...
# ))
```
    
... means nothing more than ...

```python
_temp0 = EXPR0
_temp1 = EXPR1
def FUNC():
    ...
FUNC = _temp1(FUNC)
FUNC = _temp0(FUNC)
```

```python
def my_decorator_factory(a, b, c=5, ...):
    def decorator(func):
        def wrapper(*args, **kwargs):
            ...
            return func(*args, **kwargs)
        return wrapper
    return decorator
```

```python
@my_decorator_factory('foo', 'bar')
def somefunc(...):
    ...
```

In [ ]:
def authorize_or_403(perms):
    ...
    
def require_authorized(perms):
    def decorator(func):
        def wrapper(*args, **kwargs):
            authorize_or_403(perms)
            return func(*args, **kwargs)
        return wrapper
    return decorator

@app.route('/protected')
def get_some_protected_thing():
    authorize_or_403('Patient.read')
    ...

@app.route('/protected2')
@require_authorized('Patient.read')
def get_some_other_protected_thing():
    ...

Python Tutor example:

In [40]:
import time

def timeit(prefix):
    def decorator(function):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            try:
                return function(*args, **kwargs)
            finally:
                elapsed = time.time() - start_time
                print(f'{prefix}: Took {elapsed}s')
        return wrapper
    return decorator
    
def timeit_decorator(function):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        try:
            return function(*args, **kwargs)
        finally:
            elapsed = time.time() - start_time
            print(f'Took {elapsed}s')
    return wrapper

In [41]:
@timeit('slow1')
def slow_function1(n):
    time.sleep(n)
    
@timeit_decorator
def slow_function2(n):
    time.sleep(n)
    
def slow_function3(n):
    time.sleep(n)
slow_function3 = timeit_decorator(slow_function3)

In [42]:
slow_function1(0.25)
slow_function2(2.25)
slow_function3(1)

slow1: Took 0.25247883796691895s
Took 2.2528939247131348s
Took 1.002319097518921s


# Preserving introspection

In [43]:
def document_it(func):
    # below is a nested, or inner function
    def wrapper(*args, **kwargs):
        "wrapper docstring"
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return wrapper

In [44]:
@document_it
def addit(x, y):
    """I heard I should use docstrings so here ya go"""
    return x + y

In [45]:
help(addit)

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
    wrapper docstring



In [46]:
addit

<function __main__.document_it.<locals>.wrapper(*args, **kwargs)>

In [47]:
from functools import wraps

def document_it(func):
    """This is the decorator docstring"""
    # below is a nested, or inner function
    @wraps(func)
    def wrapper(*args, **kwargs):
        """This is the wrapper docstring"""
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return wrapper

In [48]:
@document_it
def addit(x, y):
    """I heard I should use docstrings so here ya go"""
    return x + y

In [49]:
help(addit)

Help on function addit in module __main__:

addit(x, y)
    I heard I should use docstrings so here ya go



In [50]:
addit

<function __main__.addit(x, y)>

In [51]:
addit(1, 2)

Running function: addit
Positional arguments: (1, 2)
Keyword arguments: {}
Result: 3


3

In [52]:
help(document_it)

Help on function document_it in module __main__:

document_it(func)
    This is the decorator docstring



# Lab

Open the [Closures Lab][closures-lab]

[closures-lab]: ./closures-lab.ipynb
